In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from msresist.pre_processing import preprocessing
from msresist.figures.figure1 import plotClustergram

In [2]:
X = preprocessing(rawdata=True, mc_col=True)